---
author: 
  - name: Matthew Reda
    email: redam94@gmail.com
copyright: 
  holder: Matthew Reda
  year: 2024
citation: true
---

# Model Classes

> Model classes

In [ ]:
#| default_exp utils.model_classes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from abc import ABC, abstractmethod
from pathlib import Path
from collections.abc import Sequence
from typing import (
  Callable, Generic, 
  TypeVar, Union, 
  Protocol, Dict,
  TypeAlias)
import xarray as xr
import types

In [ ]:
#| export
class _Model(ABC):
  
  @abstractmethod
  def predict(self, x: xr.Dataset) -> xr.Dataset:
    pass
  
  @abstractmethod
  def contributions(self, x: xr.Dataset) -> xr.Dataset:
    pass
  
  def __repr__(self) -> str:
    return f"Model"

In [ ]:
#| export
Budget = Union[Dict[str, float], xr.Dataset]

In [ ]:
#| export
class BaseBudgetModel(_Model):
    """
    Abstract class for all models
    """
    _FUNCTION_MODULE_NAME = "model_config.py"
    
    def __init__(
      self, 
      model_name: str, # Name used to identify the model
      model_kpi: str, # Key performance indicator output by the model predict
      model_path: str|Path, # Path to the model artifact
    ):
        self.model_name: str = model_name
        self.model_kpi: str = model_kpi
        self.model_path: Path = model_path if isinstance(model_path, Path) else Path(model_path)
        self._model = self._get_model_loader()(model_path)
        self._budget_to_data = self._get_budget_to_data()

    def _get_model_loader(self) -> Callable[str|Path, _Model]:
        """
        Get the function to load the model from the path
        """
        from importlib.util import spec_from_file_location, module_from_spec
        spec = spec_from_file_location(self._FUNCTION_MODULE_NAME.replace(".py", ""), self.model_path / self._FUNCTION_MODULE_NAME)
        module = module_from_spec(spec)
        spec.loader.exec_module(module)
        return module.model_loader
    
    def _get_budget_to_data(self) -> Callable[Budget, xr.Dataset]:
        """
        Get the mapping from budget keys to data keys
        """
        from importlib.util import spec_from_file_location, module_from_spec
        spec = spec_from_file_location(self._FUNCTION_MODULE_NAME.replace(".py", ""), self.model_path / self._FUNCTION_MODULE_NAME)
        module = module_from_spec(spec)
        spec.loader.exec_module(module)
        return module.budget_to_data
    
    def predict(
        self, 
        budget: Budget # Budget
        ) -> xr.DataArray: # Predicted target variable
        """
        Predict the target variable from the input data
        """
        data = self._budget_to_data(budget, self._model)
        return self._model.predict(data)
    
    
    def contributions(
        self, 
        budget: Budget # Budget
        ) -> xr.Dataset: # Contributions of the input data to the target variable
        """
        Get the contributions of the input data to the target variable
        """
        data = self._budget_to_data(budget, self._model)
        return self._model.contributions(data)

In [ ]:
class BudgetModel(BaseBudgetModel):
    """
    Budget model class
    """
    ...

In [ ]:
m = BudgetModel("Revenue Model", "Revenue", "../../example_files/model_1")

In [ ]:
show_doc(BaseBudgetModel.predict)

---

[source](https://github.com/redam94/budget_optimizer/blob/main/budget_optimizer/utils/model_classes.py#L76){target="_blank" style="float:right; font-size:smaller"}

### BaseBudgetModel.predict

>      BaseBudgetModel.predict
>                               (budget:Union[Dict[str,float],xarray.core.datase
>                               t.Dataset])

*Predict the target variable from the input data*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| budget | Union | Budget |
| **Returns** | **DataArray** | **Predicted target variable** |

In [ ]:
m.predict({"a": 2, "b": .3})

<xarray.DataArray 'prediction' (time: 156)> Size: 1kB
array([3.55098207, 3.26499693, 3.63304337, 3.64270158, 2.90739074,
       3.07360594, 3.66742609, 3.31430186, 3.51725232, 3.1711266 ,
       3.77920098, 3.70605167, 3.46371195, 3.7785354 , 3.54045447,
       3.26706854, 3.52642969, 3.12341174, 3.6181342 , 3.45651329,
       3.53379529, 3.29016977, 3.80196174, 3.43692944, 3.44630045,
       3.37952273, 3.57605626, 3.64480026, 3.60409729, 3.69951133,
       4.12004995, 3.26377047, 3.22696829, 3.37737785, 3.7688905 ,
       3.76892156, 3.3986108 , 3.35441643, 3.16462551, 3.57667601,
       3.71602081, 3.58032556, 3.28179312, 3.50905496, 3.51446107,
       3.62896224, 3.71141772, 3.56779799, 3.51117559, 3.47202847,
       3.47717326, 3.54953481, 3.02647826, 3.34738534, 3.40316004,
       3.19716754, 3.38568241, 3.860578  , 3.20791175, 3.81149364,
       3.07700714, 3.47339759, 3.49046152, 3.5716276 , 3.64120264,
       3.69976403, 3.3771909 , 3.25843383, 3.70742347, 3.42273762,
       3.3431543 , 3.31849198, 3.15796845, 3.57481279, 3.3955854 ,
       3.60912901, 3.36718634, 3.59532238, 3.58071699, 3.32759741,
       3.44144523, 3.27116084, 3.28579301, 3.31727293, 3.08877624,
       3.58590395, 3.21343375, 3.35976148, 3.7663961 , 3.49205858,
       3.56745507, 3.57954424, 3.59361302, 3.35391484, 3.0084156 ,
       3.11663274, 3.25984424, 3.12974912, 3.55015549, 3.28159667,
       3.38914528, 3.8070451 , 3.35222724, 3.57032558, 3.19749958,
       3.38248977, 3.22818794, 3.32665854, 3.73115535, 3.10471789,
       3.58945308, 3.54903898, 3.3043232 , 3.09121336, 3.42542692,
       3.3407605 , 3.51402324, 3.63517745, 4.05530719, 3.42196392,
       3.1384508 , 3.42856926, 3.61112196, 3.87261088, 3.73028606,
       3.43765659, 3.95733743, 3.18686793, 3.21107208, 3.18260202,
       3.37301024, 3.11146916, 3.61416396, 3.38983991, 3.06700661,
       3.20603923, 3.66089566, 3.52756513, 4.04064196, 4.38988999,
       3.59440037, 3.17410705, 2.81620937, 3.55557941, 3.38427229,
       3.24152077, 3.36245886, 3.39530416, 3.7584413 , 3.42637108,
       3.3901018 , 3.29233975, 3.0823694 , 3.46791538, 3.53493548,
       4.01602692])
Coordinates:
  * time     (time) int64 1kB 1 2 3 4 5 6 7 8 ... 150 151 152 153 154 155 156

In [ ]:
show_doc(BaseBudgetModel.contributions)

---

[source](https://github.com/redam94/budget_optimizer/blob/main/budget_optimizer/utils/model_classes.py#L87){target="_blank" style="float:right; font-size:smaller"}

### BaseBudgetModel.contributions

>      BaseBudgetModel.contributions
>                                     (budget:Union[Dict[str,float],xarray.core.
>                                     dataset.Dataset])

*Get the contributions of the input data to the target variable*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| budget | Union | Budget |
| **Returns** | **Dataset** | **Contributions of the input data to the target variable** |

In [ ]:
m.contributions({"a": 2, "b": .3})

<xarray.Dataset> Size: 4kB
Dimensions:  (time: 156)
Coordinates:
  * time     (time) int64 1kB 1 2 3 4 5 6 7 8 ... 150 151 152 153 154 155 156
Data variables:
    a        (time) float64 1kB 2.802 2.45 2.93 2.986 ... 2.589 2.704 3.244
    b        (time) float64 1kB 0.7486 0.8152 0.7029 ... 0.8787 0.8312 0.7721

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()